# Data Analytics Must Knows

### Pandas, SQL

Situation: 

You are given a JSON data formatted as a string which holds basic employee information in a company that has several departments, and you are given a series of analytics questions to analyze. 

You are to provide your solutions to each question in every language/library - **Pandas**, **SQL**, and **PySpark**.

The data given to you is shown in the next cell:

In [1]:
employees_json = """
    [
        {"EmployeeID":1, "Name":"Alice", "Department":"HR", "Salary":60000, "JoiningDate":"2020-01-15", "PerformanceScore":3},
        {"EmployeeID":2, "Name":"Bob", "Department":"IT", "Salary":70000, "JoiningDate":"2019-06-20", "PerformanceScore":4},
        {"EmployeeID":3, "Name":"Charlie", "Department":"IT", "Salary":80000, "JoiningDate":"2018-07-23", "PerformanceScore":2},
        {"EmployeeID":4, "Name":"David", "Department":"HR", "Salary":65000, "JoiningDate":"2020-02-10", "PerformanceScore":5},
        {"EmployeeID":5, "Name":"Eve", "Department":"Finance", "Salary":75000, "JoiningDate":"2021-03-15", "PerformanceScore":3}
    ]
"""
bonuses_json = """
    [
        {"EmployeeID":1, "Bonus":5000},
        {"EmployeeID":2, "Bonus":7000},
        {"EmployeeID":3, "Bonus":8000},
        {"EmployeeID":6, "Bonus":6000}
    ]
"""

**Q0: Import the necessary libraries and perform the necessary setup to complete the tasks for all the indivated languages and libraries.**

Import libraries

In [2]:
# DateTime
from datetime import datetime, date 
# JSON
import json
# Pandas
import pandas as pd
# SQL
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.sql import text
# PySpark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

JSON Parsing

In [3]:
# JSON Parsing (Employee)
employees_dict = json.loads(employees_json)
employees_dict[0]

{'EmployeeID': 1,
 'Name': 'Alice',
 'Department': 'HR',
 'Salary': 60000,
 'JoiningDate': '2020-01-15',
 'PerformanceScore': 3}

In [4]:
# JSON Parsing (Bonuses)
bonuses_dict = json.loads(bonuses_json)
bonuses_dict[0]

{'EmployeeID': 1, 'Bonus': 5000}

Setup Pandas

In [5]:
employees_pd = pd.DataFrame(employees_dict)
employees_pd

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore
0,1,Alice,HR,60000,2020-01-15,3
1,2,Bob,IT,70000,2019-06-20,4
2,3,Charlie,IT,80000,2018-07-23,2
3,4,David,HR,65000,2020-02-10,5
4,5,Eve,Finance,75000,2021-03-15,3


In [6]:
bonuses_pd = pd.DataFrame(bonuses_dict)
bonuses_pd

,EmployeeID,Bonus
0,1,5000
1,2,7000
2,3,8000
3,6,6000


Setup PostgreSQL

In [7]:
# Get credentials
load_dotenv()
user = os.environ.get("USER")
pw = os.environ.get("PASS")
db = os.environ.get("DB")
host = os.environ.get("HOST")
api = os.environ.get("API")
port = 5432
schema = 'da_must_knows'

In [8]:
# Connect to database
uri = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{db}"
alchemyEngine = create_engine(uri)
conn = alchemyEngine.connect()

In [9]:
# Load to SQL
conn.execute(text(f'DROP SCHEMA IF EXISTS {schema} CASCADE'))
conn.execute(text(f'CREATE SCHEMA {schema}'))
employees_pd.to_sql(con=conn,name="employees",schema=schema)
bonuses_pd.to_sql(con=conn,name="bonuses",schema=schema)

4

In [10]:
rs = conn.execute(text(f"SELECT table_name FROM information_schema.tables WHERE table_schema='{schema}'"))
tables = [table[0] for table in rs.fetchall()]
print(f"The tables in the database are: {', '.join(tables)}")

The tables in the database are: employees, bonuses


In [11]:
for table in tables:
    print("=================================")
    print(f'Table [{table}]')
    df = pd.read_sql_query(f'SELECT * FROM {schema}.{table} LIMIT 5', conn)
    print(f'Dimensions: {df.shape[0]} rows x {df.shape[1]} columns\n')
    print(df.head())
    info_df = pd.DataFrame.from_dict({'Datatypes':df.dtypes, 'NULL count':df.isna().sum()})
    print()
    print(info_df)
    print()

Table [employees]
Dimensions: 5 rows x 7 columns

   index  EmployeeID     Name Department  Salary JoiningDate  PerformanceScore
0      0           1    Alice         HR   60000  2020-01-15                 3
1      1           2      Bob         IT   70000  2019-06-20                 4
2      2           3  Charlie         IT   80000  2018-07-23                 2
3      3           4    David         HR   65000  2020-02-10                 5
4      4           5      Eve    Finance   75000  2021-03-15                 3

                 Datatypes  NULL count
index                int64           0
EmployeeID           int64           0
Name                object           0
Department          object           0
Salary               int64           0
JoiningDate         object           0
PerformanceScore     int64           0

Table [bonuses]
Dimensions: 4 rows x 3 columns

   index  EmployeeID  Bonus
0      0           1   5000
1      1           2   7000
2      2           3   8000
3 

Setup PySpark

In [12]:
spark = SparkSession.builder.getOrCreate()

In [13]:
employees_ps = spark.createDataFrame(employees_pd)
employees_ps.show()
employees_ps.printSchema()

+----------+-------+----------+------+-----------+----------------+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|
+----------+-------+----------+------+-----------+----------------+
|         1|  Alice|        HR| 60000| 2020-01-15|               3|
|         2|    Bob|        IT| 70000| 2019-06-20|               4|
|         3|Charlie|        IT| 80000| 2018-07-23|               2|
|         4|  David|        HR| 65000| 2020-02-10|               5|
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|
+----------+-------+----------+------+-----------+----------------+

root
 |-- EmployeeID: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- JoiningDate: string (nullable = true)
 |-- PerformanceScore: long (nullable = true)



**Q1: How can you calculate the average salary for each department?**

In [14]:
# Pandas solution
average_salary = employees_pd.groupby('Department')['Salary'].mean().reset_index()
average_salary

,Department,Salary
0,Finance,75000.0
1,HR,62500.0
2,IT,75000.0


In [15]:
# SQL Solution
average_salary = pd.read_sql_query(f'''
    SELECT "Department", AVG("Salary") AS "Salary"
    FROM {schema}.employees
    GROUP BY "Department"
''',conn)
average_salary

,Department,Salary
0,Finance,75000.0
1,IT,75000.0
2,HR,62500.0


In [16]:
# PySpark Solution
employees_ps.groupBy('Department').avg("salary").alias("avg_salary").show(truncate=False)

+----------+-----------+
|Department|avg(salary)|
+----------+-----------+
|HR        |62500.0    |
|IT        |75000.0    |
|Finance   |75000.0    |
+----------+-----------+



**Q2: Determine the employee with the highest performance score in each department.**

In [17]:
# Pandas solution
x = employees_pd.groupby('Department')['PerformanceScore'].idxmax()
highest_performers = employees_pd.loc[x]
highest_performers

,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore
4,5,Eve,Finance,75000,2021-03-15,3
3,4,David,HR,65000,2020-02-10,5
1,2,Bob,IT,70000,2019-06-20,4


In [18]:
# SQL Solution
highest_performers = pd.read_sql_query(f'''
    WITH rank_per_department AS (
        SELECT *, RANK() OVER 
            (PARTITION BY "Department" ORDER BY "PerformanceScore" DESC) AS "rank"
        FROM {schema}.employees
    )
    SELECT *
    FROM rank_per_department
    WHERE rank = 1       
''',conn)
highest_performers

,index,EmployeeID,Name,Department,Salary,JoiningDate,PerformanceScore,rank
0,4,5,Eve,Finance,75000,2021-03-15,3,1
1,3,4,David,HR,65000,2020-02-10,5,1
2,1,2,Bob,IT,70000,2019-06-20,4,1


In [19]:
# PySpark Solution
windowSpec = Window.partitionBy("department").orderBy("PerformanceScore")
ranks = employees_ps.withColumn("rank",rank().over(windowSpec))
highest_performers = ranks.filter('rank == 1')
highest_performers.show()

+----------+-------+----------+------+-----------+----------------+----+
|EmployeeID|   Name|Department|Salary|JoiningDate|PerformanceScore|rank|
+----------+-------+----------+------+-----------+----------------+----+
|         5|    Eve|   Finance| 75000| 2021-03-15|               3|   1|
|         1|  Alice|        HR| 60000| 2020-01-15|               3|   1|
|         3|Charlie|        IT| 80000| 2018-07-23|               2|   1|
+----------+-------+----------+------+-----------+----------------+----+

